In [ ]:
## RUN THIS CELL IF AND ONLY IF THIS IS OPEN IN GOOGLE COLAB
!git clone https://github.com/ries-lab/SimuFLUX.git
%cd SimuFLUX/python/examples

In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np

from python.fluorophores import FlStatic
from python.psfs import PsfVectorial
from python.estimators import est_qLSQiter1D
from python.estimators import est_qLSQiter2D
from python.simulators import Simulator
from python.tools import imx

In [ ]:
fl = FlStatic(brightness=1000)  # define a static fluorophore
fl.pos = [10, 0, 0]

psf_vec = PsfVectorial() 
psf_vec.zerooffset=0.000  # True zero

sim = Simulator(fluorophores=fl)

numberOfLocalizations=1000

In [ ]:
# define scan pattern
L = 75  # size of scan pattern
orbitpoints = 6  # number of probing points in orbit
probecenter = True  # should we also probe the center?
laserpower = 5  # relative, increases brightness
pointdwelltime = 0.1  # ms, measurement time in each point
repetitions = 2 # how often to repeat the pattern scan

sim.definePattern("donut", psf_vec, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

In [ ]:
# we need an estimator. Define as component
sim.defineComponent("estdonut", "estimator", est_qLSQiter2D, parameters=[L, probecenter], dim=(0,1))

In [ ]:
seq = ["donut", "estdonut"]

out = sim.runSequence(seq, maxlocs=numberOfLocalizations)


# out.loc: localizations
# out.fluorophores: position of fluorophores
# out.raw: photon measurements
sigmaCRB=sim.calculateCRBpattern("donut", dim=(0,1))/np.sqrt(np.mean(out.loc.phot))

print("vectorial PSF:")
sim.summarize_results(out)

psf0, _ = psf_vec.imagestack("vortex")

## Pinhole
We simulate a pinhole in the detection channel.

In [ ]:
psf_vecph = PsfVectorial()
psf_vecph.setpinhole(AU=1)

In [ ]:
sim.definePattern("donut_ph", psf_vecph, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

In [ ]:
seq = ["donut_ph", "estdonut"]
print("pinhole:")
out=sim.runSequence(seq)
sim.summarize_results(out)

psfph, _ = psf_vecph.imagestack("vortex")

In [ ]:
imx([psf0, psfph])

## Misaligned pinhole
Now, lets move the pinhole (misalignment).

In [ ]:
psf_vecph2 = PsfVectorial()
psf_vecph2.setpinhole(AU=1, offset=[150, 0])

In [ ]:
sim.definePattern("donut_ph", psf_vecph2, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

In [ ]:
seq = ["donut_ph", "estdonut"]
print("pinhole misaligned:")
out=sim.runSequence(seq)
sim.summarize_results(out)

In [ ]:
# psfph2, _ = psf_vecph2.imagestack("vortex")
# imx([psf0, psfph2])

## Aberrations
Let us change the PSF by adding aberrations. Note, in this case we have to define the pattern again to calculate the PSFs anew. Instead here, we create a second PSF object.

In [ ]:
psf_vec2 = PsfVectorial()
psf_vec2.setpinhole(AU=1)

# Add Zernike:
# Zr(k,0): n, Zr(k,1): m, Zr(k,2): amplitude as fraction of wavelength
sys_aberr = {}
sys_aberr['Zr'] = np.zeros((2,3))
sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 4, 0, 0.3  # spherical aberrations 
sys_aberr['Zr'][1,0], sys_aberr['Zr'][1,1], sys_aberr['Zr'][1,2] = 2, 2, 0.05  # astigmatism 
sys_aberr['maskshift'] = [0,0]
psf_vec2.setpar(**sys_aberr)

In [ ]:
sim.definePattern("donut_aber", psf_vec2, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=4,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

seq = ["donut_aber", "estdonut"]
out = sim.runSequence(seq)

print("aberrations:")
sim.summarize_results(out)

In [ ]:
# uncomment to show PSF:
# psfab, _ = psf_vec2.imagestack("vortex")
# imx([psf0, psfab])

## Misaligned phase plate

In [ ]:
# psf_vec2 = PsfVectorial()
psf_vec2.setpinhole(AU=1)

# Add Zernike:
# Zr(k,0): n, Zr(k,1): m, Zr(k,2): amplitude as fraction of wavelength
sys_mis = {}
sys_mis['Zr'] = np.zeros((2,3))
sys_mis['Zr'][0,0], sys_mis['Zr'][0,1], sys_mis['Zr'][0,2] = 4, 0, 0.0  # spherical aberrations 
sys_mis['Zr'][1,0], sys_mis['Zr'][1,1], sys_mis['Zr'][1,2] = 2, 2, 0.0  # astigmatism 
sys_mis['maskshift'] = [0.2,0]
psf_vec2.setpar(**sys_mis)

In [ ]:
sim.definePattern("donut_misaligned", psf_vec2, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

seq = ["donut_misaligned", "estdonut"]
out = sim.runSequence(seq)

print("misaligned phase plate:")
sim.summarize_results(out)

In [ ]:
# uncomment to show PSF:
# psfab, _ = psf_vec2.imagestack("vortex")
# imx([psf0, psfab])

## Zero offset
Now, let's add an offset to the PSF to make the minium non-zero.

In [ ]:
psf_vec.zerooffset=0.005
out = sim.runSequence(seq)
print(f"zero offset = {psf_vec.zerooffset}:")
sim.summarize_results(out)

## Bead size

In [ ]:
psf_vec2.setpinhole(AU=1)

# Add Zernike:
# Zr(k,0): n, Zr(k,1): m, Zr(k,2): amplitude as fraction of wavelength
sys_b = {}
sys_b['Zr'] = np.zeros((2,3))
sys_b['Zr'][0,0], sys_b['Zr'][0,1], sys_b['Zr'][0,2] = 4, 0, 0.0  # spherical aberrations 
sys_b['Zr'][1,0], sys_b['Zr'][1,1], sys_b['Zr'][1,2] = 2, 2, 0.0  # astigmatism 
sys_b['maskshift'] = [0.0,0]
sys_b['beadradius'] = 50e-9  # 100 nm beads
psf_vec2.setpar(**sys_b)

In [ ]:
sim.definePattern("donut_bead", psf_vec2, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

seq = ["donut_bead", "estdonut"]
out = sim.runSequence(seq)

print("bead size:")
sim.summarize_results(out)

## 3D with tophat

In [ ]:
sim.fluorophores.pos = [10, 0, 20]
psf_vecth = PsfVectorial()
psf_vecth.setpinhole("AU",1)

orbitpoints = 4
probecenterxy = True
probecenterz = True
L = 75
Lz = 150

laserpower = 30

In [ ]:
sim.definePattern("tophat_xy", psf_vecth, 
                  phasemask="tophat",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenterxy, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions,
                  dim=(0,1))

sim.definePattern("tophat_z", psf_vecth, 
                  phasemask="tophat",
                  makepattern="zscan", 
                  orbitpoints=2,
                  probecenter=probecenterz, 
                  orbitL=Lz, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions,
                  dim=(2,))

In [ ]:
sim.defineComponent("esttophat_xy", "estimator", est_qLSQiter2D, parameters=[L, probecenter], dim=(0,1))
sim.defineComponent("esttophat_z", "estimator", est_qLSQiter1D, parameters=[Lz], dim=(2,))

In [ ]:
seq = ["tophat_xy", "esttophat_xy", "tophat_z", "esttophat_z"]
out=sim.runSequence(seq)

print("3D with tophat:")
sim.summarize_results(out)

## 3D with tophat and vortex

In [ ]:
laserpower=5
sim.definePattern("donut_xy", psf_vecth, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions,
                  dim=(0,1))

In [ ]:
seq = ["donut_xy", "estdonut", "tophat_z", "esttophat_z"]
out=sim.runSequence(seq)

print("3D with donut and tophat:")
sim.summarize_results(out)

## PhaseFlux 3D localization

In [ ]:
sim.fluorophores.pos = [0, 0, 0]
psf_vecphaseflux = PsfVectorial()
psf_vecphaseflux.setpinhole("AU",1)

L = 75
Lz = 150
fwhm = 450
sigmaz = 200
laserpower = 5
laserpowerz = 30
zeroposx = np.atleast_2d(np.array([-1,1,0])*L/2)
zeroposz = np.atleast_2d(np.array([-1,1,0])*Lz/2)

In [ ]:
sim.definePattern("pf_x", psf_vecphaseflux, 
                  phasemask="halfmoonx",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions,
                  dim=(0,))

sim.definePattern("pf_y", psf_vecphaseflux, 
                  phasemask="halfmoony",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions,
                  dim=(1,))

sim.definePattern("pf_z", psf_vecphaseflux, 
                  phasemask="tophat",
                  zeropos = zeroposz,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerz, 
                  repetitions=repetitions,
                  dim=(2,))

In [ ]:
sim.defineComponent("est_x", "estimator", est_qLSQiter1D, parameters=[L], dim=(0,))
sim.defineComponent("est_y", "estimator", est_qLSQiter1D, parameters=[L], dim=(1,))
sim.defineComponent("est_z", "estimator", est_qLSQiter1D, parameters=[Lz], dim=(2,))

In [ ]:
seq = ["pf_x", "est_x", "pf_y", "est_y", "pf_z", "est_z"]
out=sim.runSequence(seq)
print("PhaseFLUX:")
sim.summarize_results(out)